## imports & pathing

In [8]:
# --- Notebook setup: adjust as needed for your repo layout ---
import os, sys, json, hashlib, textwrap, time
from pathlib import Path

PROJECT_ROOT = Path.cwd()  # change if running from a subdir
SRC = PROJECT_ROOT / "src"

# Ensure we can import project modules (schema, pipeline, etc.)
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

# Optional: silence Chroma's "k>docs" info messages
os.environ.setdefault("TOKENIZERS_PARALLELISM", "false")

# Helpers
def stable_json(obj) -> str:
    """Deterministic JSON string (sorted keys, no whitespace ambiguity)."""
    return json.dumps(obj, ensure_ascii=False, sort_keys=True, separators=(",", ":"))

def sha256_text(s: str) -> str:
    return hashlib.sha256(s.encode("utf-8")).hexdigest()

def print_ok(label: str):
    print(f"✅ {label}")

def print_skip(label: str, reason: str):
    print(f"⏭️  Skipped {label}: {reason}")

import warnings
import logging


# Ignore all warnings
warnings.filterwarnings('ignore')


# Optionally, also silence specific noisy libraries
logging.getLogger('chromadb').setLevel(logging.ERROR)
logging.getLogger('openai').setLevel(logging.ERROR)
logging.getLogger('urllib3').setLevel(logging.ERROR)


## JD + resume

In [9]:
# Minimal JD & resume fixtures used in tests
JD_SIMPLE = {
    "title": "Program Manager",
    "sector": "Operations & Supply Chain",
    "location": "Hybrid – Cairo",
    "description": "We are hiring a Program Manager in Operations & Supply Chain to drive outcomes.",
    "requirements": [
        "1+ years of relevant experience",
        "Bachelor's degree or equivalent experience",
        "Proficiency in Lean",
        "Proficiency in Six Sigma",
    ],
}

RESUME_SIMPLE = textwrap.dedent("""
    Delivered 5 projects using Project Planning, SAP, Lean with measurable KPIs.
    Collaborated with 7 stakeholders to ship on schedule. Six Sigma exposure.
""").strip()


### Rules-only determinism

In [10]:
from parse_resume import parse_resume
from retrieve import build_resume_collection, retrieve_for_requirements
from scorer import score_rule_based
from schema import validate_json
from uuid import uuid4

def run_rules_once(jd: dict, resume_text: str, k: int = 3) -> dict:
    parsed = parse_resume(resume_text, jd.get("requirements", []))
    lines = parsed.get("evidence_lines", [])
    if len(lines) < 2:
        lines = [ln.strip() for ln in resume_text.splitlines() if ln.strip()]
    # Unique collection to avoid clashes; name doesn't affect scoring
    name = f"resume_v0_{uuid4().hex[:8]}"
    _, coll = build_resume_collection(lines, collection_name=name)
    reqs = [r for r in jd.get("requirements", []) if r.lower().startswith("proficiency in ")]
    hits = retrieve_for_requirements(coll, reqs, k=k)
    out = score_rule_based(jd, parsed, hits)
    ok, errs = validate_json(out)
    assert ok, f"Schema invalid: {errs}"
    return out

# Run the rules path multiple times and verify identical outputs
N = 10
outputs = []
hashes = []
overall_scores = []

for i in range(N):
    out = run_rules_once(JD_SIMPLE, RESUME_SIMPLE, k=3)
    s = stable_json(out)
    outputs.append(out)
    hashes.append(sha256_text(s))
    overall_scores.append(out["overallScore"])

print("Rules mode overallScore across runs:", overall_scores)

# Determinism check (byte-identical)
assert len(set(hashes)) == 1, "Rules mode outputs differ across runs!"
print_ok(f"Rules-only determinism: {N} runs produced identical outputs (hash={hashes[0]})")


Rules mode overallScore across runs: [76, 76, 76, 76, 76, 76, 76, 76, 76, 76]
✅ Rules-only determinism: 10 runs produced identical outputs (hash=4bf087f2610ba77e5715182d72099d120efe0e08b6c5e78a006a8ee489c6a9dc)


### LLM determinism

In [11]:
import os
from pipeline import run_pipeline, PipelineConfig
from schema import validate_json

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

def run_llm_once(jd: dict, resume_text: str, k: int = 3, model: str = "gpt-4o-mini-2024-08-06", seed: int = 42) -> dict:
    cfg = PipelineConfig(k=k, model=model, seed=seed)
    out = run_pipeline(jd, resume_text, cfg=cfg)  # has schema check & LLM repair+fallback inside
    ok, errs = validate_json(out)
    assert ok, f"Schema invalid: {errs}"
    return out

if not OPENAI_API_KEY:
    print_skip("LLM determinism", "OPENAI_API_KEY is not set; cannot call the model.")
else:
    # If your account lacks the default model, change it here to an available one.
    MODEL = "gpt-4o-mini-2024-08-06"

    M = 3
    outs_llm = []
    hashes_llm = []
    llm_overall_scores = []

    for _ in range(M):
        out = run_llm_once(JD_SIMPLE, RESUME_SIMPLE, k=3, model=MODEL, seed=42)
        s = stable_json(out)
        outs_llm.append(out)
        hashes_llm.append(sha256_text(s))
        llm_overall_scores.append(out["overallScore"])

    print("LLM mode overallScore across runs:", llm_overall_scores)

    assert len(set(hashes_llm)) == 1, "LLM mode outputs differ across runs! Check temperature/seed/model."
    print_ok(f"LLM determinism: {M} runs produced identical outputs (hash={hashes_llm[0]}) using model={MODEL}, seed=42, temp=0")


[llm] chat.completions.create(...) called
[llm] chat.completions.create(...) called
[llm] chat.completions.create(...) called


LLM mode overallScore across runs: [76, 76, 76]
✅ LLM determinism: 3 runs produced identical outputs (hash=4bf087f2610ba77e5715182d72099d120efe0e08b6c5e78a006a8ee489c6a9dc) using model=gpt-4o-mini-2024-08-06, seed=42, temp=0
